### Convolutional Neural Network Example Workflow (Keras)

This is an example of applying cnn model building based on `keras`. The data is from [kaggle MNIST Digit Recognizer Competition](https://www.kaggle.com/c/digit-recognizer)

#### 1. Preparation

Read data, preprocessing data (scale and normalize data, reshape data), set model parameters.

In [1]:
import numpy as np
import pandas as pd

In [2]:
# read data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
# check data
train.head(3)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
test.head(3)

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
train.shape, test.shape

((42000, 785), (28000, 784))

In [6]:
# split into training and validation set
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train.iloc[:, 1:].values, train.label.values, test_size=.05, random_state=218)
X_train.shape, X_val.shape

((39900, 784), (2100, 784))

In [7]:
X_test = test.values
X_test.shape

(28000, 784)

In [8]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras import backend as K

Using TensorFlow backend.


In [9]:
# setting for CNN 
batch_size, num_classes, epochs = 128, 10, 20
img_rows, img_cols = 28, 28
input_shape = (img_rows, img_cols, 1) # not channel_first

In [10]:
# reshape data into squared size
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_val = X_val.reshape(X_val.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)

In [11]:
# pre-processing: normalization
X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_val /= 255
X_test /= 255

In [12]:
# pre-processing: convert target to one-hot vector
y_train = to_categorical(y_train, num_classes)
y_val = to_categorical(y_val, num_classes)
y_train.shape, y_val.shape

((39900, 10), (2100, 10))

#### 2. Build model

Use the most common way to build a CNN using `keras`: `Sequential` API

In [13]:
def cnn_model():
    '''
    Simple CNN model for MNIST digit classification task
    conv2d -> conv2d -> maxpooling -> dropout -> flatten -> dense (fully connected) -> dropout -> dense (softmax)
    use Sequential(), there are a lot of networks that cannot use Sequential()
    '''
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(num_classes, activation='softmax'))
    
    return model

In [14]:
model = cnn_model()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs,verbose=1, validation_data=(X_val, y_val))

Train on 39900 samples, validate on 2100 samples
Epoch 1/20
39900/39900 [==============================] - 129s 3ms/step - loss: 0.2980 - acc: 0.9098 - val_loss: 0.0836 - val_acc: 0.9733
Epoch 2/20
39900/39900 [==============================] - 126s 3ms/step - loss: 0.1028 - acc: 0.9684 - val_loss: 0.0519 - val_acc: 0.9833
Epoch 3/20
39900/39900 [==============================] - 127s 3ms/step - loss: 0.0732 - acc: 0.9780 - val_loss: 0.0436 - val_acc: 0.9862
Epoch 4/20
39900/39900 [==============================] - 131s 3ms/step - loss: 0.0595 - acc: 0.9818 - val_loss: 0.0461 - val_acc: 0.9838
Epoch 5/20
39900/39900 [==============================] - 130s 3ms/step - loss: 0.0516 - acc: 0.9842 - val_loss: 0.0328 - val_acc: 0.9871
Epoch 6/20
39900/39900 [==============================] - 129s 3ms/step - loss: 0.0433 - acc: 0.9866 - val_loss: 0.0354 - val_acc: 0.9857
Epoch 7/20
39900/39900 [==============================] - 125s 3ms/step - loss: 0.0365 - acc: 0.9881 - val_loss: 0.0320 - v

In [15]:
score = model.evaluate(X_val, y_val, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.0318565379559
Test accuracy: 0.988571428571


In [16]:
# probabilities
pred = model.predict(X_test)
pred_label = pred.argmax(axis=-1)

In [17]:
# submission
submission = pd.DataFrame({'ImageId': np.arange(1, test.shape[0] + 1), 
                           'Label': pred_label})
submission.to_csv('submission_simple_cnn.csv', index=False)

In [18]:
def lenet():
    '''
    LeNet
    conv2d -> maxpooling -> conv2d -> maxpooling -> dense -> dropout -> dense -> dropout -> dense (softmax)
    '''
    model = Sequential()
    model.add(Conv2D(filters=12, kernel_size=(5, 5), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=25, kernel_size=(5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(180, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(num_classes, activation='softmax'))
    
    return model

In [19]:
lenet_model = lenet()
lenet_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
lenet_model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs + 50,verbose=1, validation_data=(X_val, y_val))

Train on 39900 samples, validate on 2100 samples
Epoch 1/70
39900/39900 [==============================] - 21s 522us/step - loss: 0.6180 - acc: 0.8004 - val_loss: 0.1181 - val_acc: 0.9610
Epoch 2/70
39900/39900 [==============================] - 20s 495us/step - loss: 0.1748 - acc: 0.9514 - val_loss: 0.0744 - val_acc: 0.9762
Epoch 3/70
39900/39900 [==============================] - 20s 496us/step - loss: 0.1234 - acc: 0.9655 - val_loss: 0.0614 - val_acc: 0.9810
Epoch 4/70
39900/39900 [==============================] - 20s 496us/step - loss: 0.0982 - acc: 0.9735 - val_loss: 0.0482 - val_acc: 0.9843
Epoch 5/70
39900/39900 [==============================] - 20s 494us/step - loss: 0.0839 - acc: 0.9769 - val_loss: 0.0452 - val_acc: 0.9862
Epoch 6/70
39900/39900 [==============================] - 20s 495us/step - loss: 0.0720 - acc: 0.9817 - val_loss: 0.0425 - val_acc: 0.9852
Epoch 7/70
39900/39900 [==============================] - 20s 496us/step - loss: 0.0630 - acc: 0.9823 - val_loss: 0.0

39900/39900 [==============================] - 22s 540us/step - loss: 0.0092 - acc: 0.9972 - val_loss: 0.0605 - val_acc: 0.9886
Epoch 60/70
39900/39900 [==============================] - 22s 540us/step - loss: 0.0090 - acc: 0.9973 - val_loss: 0.0735 - val_acc: 0.9876
Epoch 61/70
39900/39900 [==============================] - 22s 541us/step - loss: 0.0101 - acc: 0.9969 - val_loss: 0.0675 - val_acc: 0.9871
Epoch 62/70
39900/39900 [==============================] - 22s 541us/step - loss: 0.0081 - acc: 0.9979 - val_loss: 0.0601 - val_acc: 0.9876
Epoch 63/70
39900/39900 [==============================] - 22s 539us/step - loss: 0.0101 - acc: 0.9969 - val_loss: 0.0540 - val_acc: 0.9890
Epoch 64/70
39900/39900 [==============================] - 22s 553us/step - loss: 0.0094 - acc: 0.9971 - val_loss: 0.0752 - val_acc: 0.9886
Epoch 65/70
39900/39900 [==============================] - 22s 557us/step - loss: 0.0069 - acc: 0.9979 - val_loss: 0.0681 - val_acc: 0.9890
Epoch 66/70
39900/39900 [=======

In [20]:
pred = lenet_model.predict(X_test)
pred_label = pred.argmax(axis=-1)

In [21]:
submission['Label'] = pred_label
submission.to_csv('submission_lenet_seq.csv', index=False)

Use `Functional` API

In [22]:
def lenet2(input_shape):
    '''
    Another way to defind a CNN, use Model()
    '''
    X_input = Input(input_shape)
    X = Conv2D(64, (5, 5), activation='relu')(X_input)
    X = MaxPooling2D((2, 2))(X)
    X = Conv2D(128, (5, 5), activation='relu')(X)
    X = MaxPooling2D((2, 2))(X)
    X = Flatten()(X)
    X = Dense(256, activation='relu')(X)
    X = Dropout(.5)(X)
    X = Dense(100, activation='relu')(X)
    X = Dropout(.5)(X)
    X = Dense(10, activation='softmax')(X)
    
    model = Model(inputs=[X_input], outputs=X)
    return model

In [23]:
lenet_model2 = lenet()
lenet_model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
lenet_model2.fit(X_train, y_train, batch_size=batch_size, epochs=epochs + 30,verbose=1, validation_data=(X_val, y_val))

Train on 39900 samples, validate on 2100 samples
Epoch 1/50
39900/39900 [==============================] - 22s 563us/step - loss: 0.5998 - acc: 0.8063 - val_loss: 0.1004 - val_acc: 0.9671
Epoch 2/50
39900/39900 [==============================] - 22s 549us/step - loss: 0.1642 - acc: 0.9539 - val_loss: 0.0654 - val_acc: 0.9771
Epoch 3/50
39900/39900 [==============================] - 20s 506us/step - loss: 0.1160 - acc: 0.9688 - val_loss: 0.0546 - val_acc: 0.9790
Epoch 4/50
39900/39900 [==============================] - 20s 501us/step - loss: 0.0898 - acc: 0.9754 - val_loss: 0.0436 - val_acc: 0.9862
Epoch 5/50
39900/39900 [==============================] - 22s 554us/step - loss: 0.0745 - acc: 0.9790 - val_loss: 0.0461 - val_acc: 0.9852
Epoch 6/50
39900/39900 [==============================] - 22s 548us/step - loss: 0.0679 - acc: 0.9818 - val_loss: 0.0473 - val_acc: 0.9843
Epoch 7/50
39900/39900 [==============================] - 22s 548us/step - loss: 0.0578 - acc: 0.9838 - val_loss: 0.0

In [24]:
pred = lenet_model2.predict(X_test)
pred_label = pred.argmax(axis=-1)

In [25]:
submission['Label'] = pred_label
submission.to_csv('submission_lenet_model.csv', index=False)